In [1]:
import torch, torchvision, torch.nn as nn, json, os
from collections import defaultdict
from pathlib import Path

# ---------- 1. Build the exact network definition ----------
def make_vgg16():
    net = torchvision.models.vgg16_bn(pretrained=False)
    net.features[0] = nn.Conv2d(3, 64, kernel_size=3, padding=1)
    net.avgpool = nn.AdaptiveAvgPool2d((1, 1))
    net.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(512, 512), nn.ReLU(True), nn.Dropout(),
        nn.Linear(512, 512), nn.ReLU(True), nn.Dropout(),
        nn.Linear(512, 10),
    )
    return net

# ---------- 2. Load baseline & faulty checkpoints ----------
device = "cpu"
base   = make_vgg16().to(device)
faulty = make_vgg16().to(device)

base.load_state_dict(torch.load("model/baseline_vgg16.pt", map_location=device))
faulty.load_state_dict(torch.load("model/faulty_vgg16.pt",  map_location=device))

# ---------- 3. Detect changed filters ----------
layer_filters = defaultdict(list)   # {layer_idx: [filter_idx, …]}
total_filters = 0
tol = 1e-7

for idx, (b, f) in enumerate(zip(base.modules(), faulty.modules())):
    if isinstance(b, nn.Conv2d):
        for fi in range(b.weight.size(0)):
            total_filters += 1
            if not torch.allclose(b.weight[fi], f.weight[fi], atol=tol, rtol=0):
                layer_filters[idx].append(fi)

# ---------- 4. Save as pretty JSON ----------
os.makedirs("content", exist_ok=True)
outfile = Path("content/faulty_filter_index.json")

# convert keys to strings so JSON is explicit ("layer 3": [...])
json_ready = {f"layer {k}": v for k, v in layer_filters.items()}

outfile.write_text(json.dumps(json_ready, indent=2))
print(f"\nSaved changed-filter index to: {outfile.resolve()}\n")

# ---------- 5. Console summary ----------
print(f"Total Conv2d filters examined : {total_filters}")
print(f"Filters modified by injection : {sum(len(v) for v in layer_filters.values())}\n")

for layer, flist in sorted(layer_filters.items()):
    print(f"layer {layer}:")
    for fi in flist:
        print(f"  filter {fi}")


/home/farzin/anaconda3/envs/hp_dnn/lib/python3.6/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Saved changed-filter index to: /home/farzin/Project/Tolerative_DNN/Filter base/content/faulty_filter_index.json

Total Conv2d filters examined : 4224
Filters modified by injection : 1682

layer 2:
  filter 2
  filter 6
  filter 8
  filter 11
  filter 14
  filter 16
  filter 18
  filter 21
  filter 23
  filter 25
  filter 26
  filter 30
  filter 36
  filter 37
  filter 42
  filter 43
  filter 47
  filter 48
  filter 49
  filter 50
  filter 51
  filter 53
  filter 57
  filter 59
  filter 61
layer 5:
  filter 0
  filter 1
  filter 2
  filter 5
  filter 9
  filter 13
  filter 16
  filter 17
  filter 18
  filter 25
  filter 27
  filter 28
  filter 34
  filter 36
  filter 39
  filter 40
  filter 43
  filter 45
  filter 47
  filter 50
  filter 53
  filter 55
  filter 56
  filter 59
  filter 61
layer 9:
  filter 0
  filter 9
  filter 10
  filter 11
  filter 15
  filter 16
  filter 20
  filter 22
  filter 24
  filter 25
  filter 26
  filter 27
  filter 29
  filter 31
  filter 37
  filter 38
  